In [2]:
!pip install datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system 

In [3]:
from datasets import load_dataset

ds = load_dataset("datablations/c4-filter-small", split="train")     # Dòng 3
ds = ds.select_columns(["text"])                                     # Dòng 4
ds = ds.train_test_split(test_size=0.1)                              # Dòng 5


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/791 [00:00<?, ?B/s]

(…)-00000-of-00001-091e566583af27e4.parquet:   0%|          | 0.00/141M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [4]:
!pip install tokenizers


In [5]:
# 1. Tải bộ dữ liệu C4-filter nhỏ
from datasets import load_dataset
ds = load_dataset("datablations/c4-filter-small", split="train")
ds = ds.select_columns(["text"])
ds = ds.train_test_split(test_size=0.1)

# 2. Import các thành phần của Tokenizer
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import ByteLevel
from tokenizers.normalizers import NFKC
from tokenizers.decoders import ByteLevel as ByteLevelDecoder

# 3. Khởi tạo tokenizer
tokenizer = Tokenizer(BPE())
tokenizer.pre_tokenizer = ByteLevel()
tokenizer.normalizer = NFKC()
tokenizer.decoder = ByteLevelDecoder()

# 4. Cấu hình trainer với vocab_size = 50257 (chuẩn GPT-2)
trainer = BpeTrainer(
    vocab_size=50257,
    special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"]
)

# 5. Huấn luyện tokenizer từ text
tokenizer.train_from_iterator(ds["train"]["text"], trainer)

# 6. Lưu tokenizer
tokenizer.save("gpt_tokenizer.json")

# 7. (Tuỳ chọn) Kiểm tra tokenizer mới huấn luyện
output = tokenizer.encode("I am learning to pretrain GPT2 from scratch.")
print("Token IDs:", output.ids)
print("Tokens:", output.tokens)


Token IDs: [258, 607, 2319, 233, 2453, 5525, 10108, 54, 22, 369, 9046, 18]
Tokens: ['ĠI', 'Ġam', 'Ġlearning', 'Ġto', 'Ġpret', 'rain', 'ĠGP', 'T', '2', 'Ġfrom', 'Ġscratch', '.']


In [8]:
def tokenize(example):
    encoded_batch = tokenizer.encode_batch(example["text"])
    return {
        "input_ids": [e.ids for e in encoded_batch],
        "attention_mask": [e.attention_mask for e in encoded_batch]
    }

tokenized_ds = ds.map(
    tokenize,
    remove_columns=["text"],
    batched=True,
    num_proc=20
)


Map (num_proc=20):   0%|          | 0/90000 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [9]:
block_size = 512

def group_texts(examples):
    # concat input_ids
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated["input_ids"])
    total_length = (total_length // block_size) * block_size

    # split block_size
    result = {
        k: [concatenated[k][i : i + block_size] for i in range(0, total_length, block_size)]
        for k in concatenated
    }

    # prepare labels
    result["labels"] = result["input_ids"].copy()
    return result

lm_ds = tokenized_ds.map(group_texts, batched=True, num_proc=20)


Map (num_proc=20):   0%|          | 0/90000 [00:00<?, ? examples/s]

Map (num_proc=20):   0%|          | 0/10000 [00:00<?, ? examples/s]